In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
from bs4 import BeautifulSoup
import threading
from concurrent.futures import ThreadPoolExecutor

In [40]:
project_list = pd.read_excel('project_5000Up.xlsx')
project_list = project_list['Project URL'].tolist()
project_list = project_list[:100]

In [41]:
# Headless mode for Chrome
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-renderer-backgrounding")
chrome_options.add_argument("--disable-background-timer-throttling")
chrome_options.add_argument("--disable-backgrounding-occluded-windows")
chrome_options.add_argument("--disable-client-side-phishing-detection")
chrome_options.add_argument("--disable-crash-reporter")
chrome_options.add_argument("--disable-oopr-debug-crash-dump")
chrome_options.add_argument("--no-crash-upload")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-low-res-tiling")
chrome_options.add_argument("--log-level=3")
chrome_options.add_argument("--silent")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")

#proxy_server_url = "104.251.224.95"
#chrome_options.add_argument(f'--proxy-server={proxy_server_url}')

In [42]:
def scrape_prs(project_url):
    driver = webdriver.Chrome(options=chrome_options)
    project = project_url[19:]
    # Pull Requests
    pull_url = project_url + "/pulls"
    
    for i in range(0,10):
        driver.get(pull_url)
        # Wait for the document to be in 'complete' state
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.TAG_NAME, 'body'))
        )
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        
        open_prs = soup.find(href=f"/{project}/pulls?q=is%3Aopen+is%3Apr")
        close_prs = soup.find(href=f"/{project}/pulls?q=is%3Apr+is%3Aclosed")
        if open_prs != None and close_prs != None:
            open_prs = open_prs.text.split()[0]
            close_prs = close_prs.text.split()[0]
            print(f"{project_url}: {open_prs} open_prs and {close_prs} close_prs")
            driver.quit()
            return [project_url, open_prs, close_prs]
        else:
            time.sleep(10)
    driver.quit()
    print(f"{project_url}: open_prs and close_prs not found")
    return [project_url, None, None]

In [43]:
project_prs = []
with ThreadPoolExecutor(max_workers=10) as p:
    features = p.map(scrape_prs, project_list)
    for f in features:
        project_prs.append(f)

https://github.com/996icu/996.ICU: 3 open_prs and 1,970 close_prs
https://github.com/jwasham/coding-interview-university: 11 open_prs and 936 close_prs
https://github.com/kamranahmedse/developer-roadmap: 312 open_prs and 2,669 close_prs
https://github.com/donnemartin/system-design-primer: 219 open_prs and 300 close_prs
https://github.com/codecrafters-io/build-your-own-x: 137 open_prs and 240 close_prs
https://github.com/EbookFoundation/free-programming-books: 14 open_prs and 9,853 close_prs
https://github.com/facebook/react: 435 open_prs and 14,236 close_prs
https://github.com/public-apis/public-apis: 243 open_prs and 2,880 close_prs
https://github.com/sindresorhus/awesome: 23 open_prs and 1,981 close_prs
https://github.com/freeCodeCamp/freeCodeCamp: 70 open_prs and 35,644 close_prs
https://github.com/vuejs/vue: 248 open_prs and 2,264 close_prs
https://github.com/vinta/awesome-python: 401 open_prs and 1,488 close_prs
https://github.com/trekhleb/javascript-algorithms: 221 open_prs and 5

In [44]:
def scrape_owner(project_url):
    driver = webdriver.Chrome(options=chrome_options)    
    project = project_url[19:]
    creator = project.split('/')[0]
    
    # Verified Repo Owner
    owner_url = f"https://github.com/{creator}"
    driver.get(owner_url)

    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )    

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    #print(soup.prettify())

    verified = soup.find('summary', {'title': 'Label: Verified'})
    if verified != None:
        verified = verified.text.split()[0]
        
    print(f"{project_url} owner status: {verified}")

    # Number of Owner Followers
    '''
    for i in range(0,2):
        followers = soup.find('a', class_='Link--secondary no-underline no-wrap')
        if followers != None:
            followers = followers.text.split()[0]
            break
        else:
            time.sleep(10)
            driver.get(owner_url)
            # Wait for the document to be in 'complete' state
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.TAG_NAME, 'body'))
            )    
            
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
    '''
    followers = soup.find('a', class_='Link--secondary no-underline no-wrap')
    if followers != None:
        followers = followers.text.split()[0]
    print(f"{project_url} followers: {followers}")

    # Number of Owner Members
    members = soup.find('span', class_='Counter js-profile-member-count')

    if members != None:
        while members.text == "":
            driver.get(owner_url)
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.TAG_NAME, 'body'))
            )    
        
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            time.sleep(5)
            members = soup.find('span', class_='Counter js-profile-member-count')
        members = members.text.split()[0]
        
    print(f"{project_url} members: {members}")

    # Number of Other Repositories by Owner
    repositories = soup.find('span', class_='Counter js-profile-repository-count')
    if repositories != None:
        repositories = repositories.text.split()[0]
    print(f"{project_url} repositories: {repositories}")
    
    driver.quit()
    return [project_url, verified, followers, members, repositories]

In [45]:
project_owner = []
with ThreadPoolExecutor(max_workers=10) as p:
    features = p.map(scrape_owner, project_list)
    for f in features:
        project_owner.append(f)

https://github.com/freeCodeCamp/freeCodeCamp owner status: Verified
https://github.com/freeCodeCamp/freeCodeCamp followers: 21.8k
https://github.com/freeCodeCamp/freeCodeCamp members: 82
https://github.com/freeCodeCamp/freeCodeCamp repositories: 220
https://github.com/public-apis/public-apis owner status: None
https://github.com/public-apis/public-apis followers: None
https://github.com/public-apis/public-apis members: None
https://github.com/public-apis/public-apis repositories: 1
https://github.com/EbookFoundation/free-programming-books owner status: None
https://github.com/EbookFoundation/free-programming-books followers: 6.7k
https://github.com/EbookFoundation/free-programming-books members: 10
https://github.com/EbookFoundation/free-programming-books repositories: 33
https://github.com/facebook/react owner status: Verified
https://github.com/facebook/react followers: 21.7k
https://github.com/facebook/react members: 111
https://github.com/facebook/react repositories: 135
https://gi

In [46]:
def scrape_insight(project_url):
    driver = webdriver.Chrome(options=chrome_options)
    project = project_url[19:]
    creator = project.split('/')[0]
    
    # Active prs and active issues
    insight_url = f"{project_url}/pulse"
    driver.get(insight_url)

    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )    
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    active = soup.find_all('div', class_='mt-2')
    active_prs = active[0]
    active_issues = active[1]

    if active_prs != None:
        active_prs = active_prs.text.split()[0]

    if active_issues != None:
        active_issues = active_issues.text.split()[0]

    print(f"{project_url}: {active_prs} Active pull requests, {active_issues} Active issues")
    return [project_url, active_prs, active_issues]

In [47]:
project_insight = []
with ThreadPoolExecutor(max_workers=10) as p:
    features = p.map(scrape_insight, project_list)
    for f in features:
        project_insight.append(f)

https://github.com/996icu/996.ICU: 0 Active pull requests, 0 Active issues
https://github.com/kamranahmedse/developer-roadmap: 18 Active pull requests, 20 Active issues
https://github.com/donnemartin/system-design-primer: 0 Active pull requests, 0 Active issues
https://github.com/public-apis/public-apis: 6 Active pull requests, 19 Active issues
https://github.com/jwasham/coding-interview-university: 0 Active pull requests, 0 Active issues
https://github.com/EbookFoundation/free-programming-books: 4 Active pull requests, 1 Active issues
https://github.com/sindresorhus/awesome: 2 Active pull requests, 1 Active issues
https://github.com/codecrafters-io/build-your-own-x: 2 Active pull requests, 1 Active issues
https://github.com/facebook/react: 37 Active pull requests, 17 Active issues
https://github.com/freeCodeCamp/freeCodeCamp: 62 Active pull requests, 30 Active issues
https://github.com/vuejs/vue: 0 Active pull requests, 2 Active issues
https://github.com/getify/You-Dont-Know-JS: 0 Act

In [48]:
####### IDEA: Merge Two Pandas DataFrames using Outer Join Merge ###########
# Potential Scrapes:
# ../issues => Open/closed issues, # of labels, # of milestones
# ../pulls
# ../actions => # of workflow runs
# ../pulse => Active pr and Active issues
# ../network/dependencies
# Sponsered
# Watches
# 
# Ones in Bash Script:
# Commits
# Tags/Releases
# Branches
# Languages
# Contributors

def scrape_page(project_url):

    project_features = []
    
    # Add url to list
    project_features.append(project_url)
    
    # Get the OWNER/REPO
    project = project_url[19:]

    # Set up Web Driver
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(project_url)

    # Get number of watches and sponsors
    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )
    
    # Parse HTML
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    
    num_watches = soup.find(href=f"/{project}/watchers").find("strong").text
    
    creator = project.split('/')[0]
    sponsored = "Yes" if soup.find(href=f"/sponsors/{creator}") != None else "No"
    
    if sponsored == "Yes":
        driver.get(f"https://github.com/sponsors/{creator}")
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.TAG_NAME, 'body'))
        )
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        
        current_sponsors = soup.find(lambda tag: tag.name == 'h4' and 'Current sponsors' in tag.get_text())
        past_sponsors = soup.find(lambda tag: tag.name == 'h4' and 'Past sponsors' in tag.get_text())

        if current_sponsors == None:
            current_sponsors = soup.find('p', class_='f3-light color-fg-muted mb-3')
            current_sponsors = current_sponsors.text.split()[0]
            past_sponsors = 0
        else:
            current_sponsors = current_sponsors.text.split()[2]
            past_sponsors = past_sponsors.text.split()[2]

    else:
        current_sponsors = 0
        past_sponsors = 0

    print(f"{project_url}: {current_sponsors} Current sponsors, {past_sponsors} Past sponsors")
    project_features.append(sponsored)
    project_features.append(current_sponsors)
    project_features.append(past_sponsors)

    project_features.append(num_watches)
    
    # Issues
    issue_url = project_url + "/issues"
    driver.get(issue_url)

    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )
    
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")

    open_issues = None if soup.find(href=f"/{project}/issues?q=is%3Aopen+is%3Aissue") == None else soup.find(href=f"/{project}/issues?q=is%3Aopen+is%3Aissue").text.split()[0]
    closed_issues = None if soup.find(href=f"/{project}/issues?q=is%3Aissue+is%3Aclosed") == None else soup.find(href=f"/{project}/issues?q=is%3Aissue+is%3Aclosed").text.split()[0]
    num_labels = None if soup.find(href=f"/{project}/labels") == None else soup.find(href=f"/{project}/labels").find("span").text
    num_milestones = None if soup.find(href=f"/{project}/milestones") == None else soup.find(href=f"/{project}/milestones").find("span").text


    project_features.append(open_issues)
    project_features.append(closed_issues)
    project_features.append(num_labels)
    project_features.append(num_milestones)

    print(f"{project_url}, Open issues: {open_issues}, Closed issues: {closed_issues}")

    # Number of Workflow Runs
    workflow_url = project_url + "/actions"
    driver.get(workflow_url)

    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    workflow = soup.find(lambda tag: tag.name == 'strong' and 'workflow runs' in tag.get_text())
    if workflow != None:
        workflow = workflow.text.split()[0]

    project_features.append(workflow)
    
    # Number of Dependent Repos
    dependent_url = project_url + "/network/dependents"
    driver.get(dependent_url)

    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    dependents = soup.find('a', class_='btn-link selected')
    if dependents != None:
        dependents = dependents.text.split()[0]
    
    project_features.append(dependents)
    print(f"{project_url} {dependents} dependents, {workflow} workflows")
        
    # Clean close the Web Session and window(s)
    driver.quit()

    return project_features

In [49]:
#df = pd.read_excel('projects.xlsx') # can also index sheet by name or fetch all sheets
#project_list = df['Project'].tolist()
#project_list = ["https://github.com/twbs/bootstrap", "https://github.com/freeCodeCamp/freeCodeCamp"]
#for p in project_list[:10]:
#    scrape_page(p)
projects = []
# Declare list to be used to store projects that failed to collect any information
error_projects = []
#for project in project_list:
#    result = scrape_page(project)
#    projects.append(result)
low = 0
high = 10
with ThreadPoolExecutor(max_workers=10) as p:
    while high <= len(project_list):
        features = p.map(scrape_page, project_list[low:high])
        for f in features:
            if f[0] != None:
                projects.append(f)
            else:
                error_projects.append(f[1])

        low += 10
        high += 10

https://github.com/996icu/996.ICU: 0 Current sponsors, 0 Past sponsors
https://github.com/996icu/996.ICU, Open issues: None, Closed issues: None
https://github.com/facebook/react: 0 Current sponsors, 0 Past sponsors
https://github.com/codecrafters-io/build-your-own-x: 0 Current sponsors, 0 Past sponsors
https://github.com/kamranahmedse/developer-roadmap: 13 Current sponsors, 124 Past sponsors
https://github.com/EbookFoundation/free-programming-books: 14 Current sponsors, 45 Past sponsors
https://github.com/sindresorhus/awesome: 264 Current sponsors, 1,162 Past sponsors
https://github.com/donnemartin/system-design-primer: 0 Current sponsors, 0 Past sponsors
https://github.com/facebook/react, Open issues: 1,182, Closed issues: 11,568
https://github.com/freeCodeCamp/freeCodeCamp: 56 Current sponsors, 161 Past sponsors
https://github.com/kamranahmedse/developer-roadmap, Open issues: 767, Closed issues: 1,070
https://github.com/996icu/996.ICU 25 dependents, None workflows
https://github.com

while len(error_projects) != 0:
    print(len(error_projects))
    with ThreadPoolExecutor(max_workers=10) as p:
        features = p.map(scrape_page, error_projects)
        error_projects = []
        for f in features:
            if f[0] != None:
                projects.append(f)
            else:
                error_projects.append(f[1])

In [50]:
projects_df = pd.DataFrame(projects, columns=['Project URL', 
                                              'Sponsored',
                                              'Current Sponsors',
                                              'Past Sponsors',
                                              'Number of Watches',
                                              'Open Issues',
                                              'Closed Issues',
                                              'Number of Labels',
                                              'Number of Milestones',
                                              'Number of Workflow Runs',
                                              'Number of Dependents',
                                             ])

In [51]:
insight_df = pd.DataFrame(project_insight, columns=['Project URL', 'Active Pull Requests', 'Active Issues'])
prs_df = pd.DataFrame(project_prs, columns=['Project URL', 'Open Pull Requests', 'Closed Pull Requests'])
owner_df = pd.DataFrame(project_owner, columns=['Project URL', 'Verified Owner', 'Followers of Owner', 'Members of Owner', 'Repos of Owner'])
merged = pd.merge(insight_df, prs_df, on='Project URL', how='outer')
merged = pd.merge(merged, owner_df, on='Project URL', how='outer')
merged = pd.merge(merged, projects_df, on='Project URL', how='outer')

In [52]:
merged

,Project URL,Active Pull Requests,Active Issues,Open Pull Requests,Closed Pull Requests,Verified Owner,Followers of Owner,Members of Owner,Repos of Owner,Sponsored,Current Sponsors,Past Sponsors,Number of Watches,Open Issues,Closed Issues,Number of Labels,Number of Milestones,Number of Workflow Runs,Number of Dependents
0,https://github.com/521xueweihan/HelloGitHub,0,11,4,36,None,8.4k,None,None,Yes,10,0,2.9k,107,"2,495",29,0,None,19
1,https://github.com/996icu/996.ICU,0,0,3,"1,970",None,8.2k,None,None,No,0,0,4.2k,None,None,14,3,None,25
2,https://github.com/AUTOMATIC1111/stable-diffus...,35,44,13,"2,818",None,12.8k,None,None,No,0,0,1k,None,None,None,None,"9,517",19
3,https://github.com/Chalarangelo/30-seconds-of-...,0,0,0,"1,604",None,2.6k,None,None,No,0,0,2.6k,None,None,None,None,"1,184",108
4,https://github.com/CyC2018/CS-Notes,0,0,63,547,None,14.4k,None,None,No,0,0,5.3k,130,446,15,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://github.com/vercel/next.js,123,85,416,"20,342",Verified,13.9k,67,142,No,0,0,1.4k,None,None,None,None,"150,228","2,497,731"
96,https://github.com/vinta/awesome-python,1,0,401,"1,488",None,7.4k,None,None,No,0,0,6k,None,None,8,0,None,21
97,https://github.com/vuejs/vue,0,2,248,"2,264",Verified,14.4k,65,123,No,0,0,5.9k,356,"9,659",36,0,69,0
98,https://github.com/yangshun/tech-interview-han...,0,0,7,526,None,10k,None,None,Yes,2,15,2k,None,None,None,None,26,18


try:
    with pd.ExcelWriter(
        "project_HTMLfeatures.xlsx",
        mode="a",
        engine="openpyxl",
        if_sheet_exists="overlay",
    ) as writer:
         projects_df.to_excel(writer,sheet_name="Sheet1", startrow=writer.sheets["Sheet1"].max_row, index = False,header= False)
except FileNotFoundError:
    projects_df.to_excel("project_HTMLfeatures.xlsx", index=False)